In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# read in all the words
words = open('names.txt', 'r').read().splitlines()
print(words[:8])
print(len(words))

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']
32033


In [3]:
# build the vocab of characters and mapping to and from the integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [857]:
# building the dataset
block_size = 8

def build_dataset(words):
    X, Y = [], []

    for w in words:
        # print(w)
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]  # crop and append

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])  # 80%
Xdev, Ydev = build_dataset(words[n1:n2])  # 10%
Xte, Yte = build_dataset(words[n2:])  # 10%
X, Y = build_dataset(words)

torch.Size([182512, 8]) torch.Size([182512])
torch.Size([22860, 8]) torch.Size([22860])
torch.Size([22774, 8]) torch.Size([22774])
torch.Size([228146, 8]) torch.Size([228146])


In [ ]:
# === === === === === === === === === === === === === === ===
class Linear:

    def __init__(self, fan_in, fan_out, bias=True):
        self.weight = torch.randn((fan_in, fan_out)) / fan_in**0.5  # kaiming init
        self.bias = torch.randn(fan_out) if bias else None

    def __call__(self, x):
        self.out = x @ self.weight
        if self.bias is not None:
            self.out += self.bias
        return self.out
    
    def parameters(self):
        return [self.weight] + ([] if self.bias is None else [self.bias])

# === === === === === === === === === === === === === === ===
class BatchNorm1d:

    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.eps = eps
        self.momentum = momentum
        self.training = True
        # parameters (trained with backprop)
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)
        # buffers (trained with a running 'momentum update')
        self.running_mean = torch.zeros(dim)
        self.running_var = torch.ones(dim)

    def __call__(self, x):
        if self.training:
            if x.ndim == 2:
                dim = 0
            elif x.ndim == 3:
                dim = (0, 1)
            xmean = x.mean(0, keepdim=True)  # batch mean
            xvar = x.var(0, keepdim=True)  # batch variance
            # update buffer
            with torch.no_grad():
                self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * xmean
                self.running_var = (1 - self.momentum) * self.running_var + self.momentum * xvar
        else:
            xmean = self.running_mean
            xvar = self.running_var
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps)  # normalization
        self.out = self.gamma * xhat + self.beta            
        return self.out
    
    def parameters(self):
        return [self.gamma, self.beta]
    
# === === === === === === === === === === === === === === ===
class Tanh:

    def __call__(self, x):
        self.out = torch.tanh(x)
        return self.out

    def parameters(self):
        return []
    
# === === === === === === === === === === === === === === ===
class Embedding:

    def __init__(self, num_embeddings, embedding_dim):
        self.weight = torch.randn((num_embeddings, embedding_dim))

    def __call__(self, IX):
        self.out = self.weight[IX]
        return self.out
    
    def parameters(self):
        return [self.weight]

# === === === === === === === === === === === === === === ===
class FlattenConsecutive:

    def __init__(self, n):
        self.n = n

    def __call__(self, x):
        B, T, C = x.shape
        x = x.view(B, T//self.n, C*self.n)
        if x.shape[1] == 1:
            x = x.squeeze(1)
        self.out = x
        return self.out
    
    def parameters(self):
        return []
    
# === === === === === === === === === === === === === === ===
class Sequential:

    def __init__(self, layers):
        self.layers = layers

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        self.out = x
        return self.out
    
    def parameters(self):
        # get parameters of all layers and stretch them out into one list
        return [p for layer in self.layers for p in layer.parameters()]

torch.manual_seed(42);

In [865]:
n_embd = 24
n_hidden = 128

model = Sequential([
    Embedding(vocab_size, n_embd), 
    FlattenConsecutive(2), Linear(n_embd * 2, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
    FlattenConsecutive(2), Linear(n_hidden * 2, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
    FlattenConsecutive(2), Linear(n_hidden * 2, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
    Linear(n_hidden, vocab_size),
])

# parameter init
with torch.no_grad():
    model.layers[-1].weight *= 0.1  # make last layer less confident

parameters = model.parameters()
print(sum(p.nelement() for p in parameters))
for p in parameters:
    p.requires_grad = True

76579


In [866]:
max_steps = 200000
batch_size = 64
lossi = []

for i in range(max_steps):

    # minibatch construct
    ix = torch.randint(0, Xtr.shape[0], (batch_size,))
    Xb, Yb = Xtr[ix], Ytr[ix]

    # forward pass
    logits = model(Xb)
    loss = F.cross_entropy(logits , Yb)

    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()

    # update: simple SGD
    lr = 0.1 if i < 150000 else 0.01  # step learning rate decay
    for p in parameters:
        p.data -= lr * p.grad

    # track stats
    if i % 10000 == 0:  # print every once in a while
        print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
    lossi.append(loss.log10().item())

    break

      0/ 200000: 3.8526


In [867]:
for layer in model.layers:
    print(layer.__class__.__name__, ':', tuple(layer.out.shape))

Embedding : (64, 8, 24)
FlattenConsecutive : (64, 4, 48)
Linear : (64, 4, 128)
BatchNorm1d : (64, 4, 128)
Tanh : (64, 4, 128)
FlattenConsecutive : (64, 2, 256)
Linear : (64, 2, 128)
BatchNorm1d : (64, 2, 128)
Tanh : (64, 2, 128)
FlattenConsecutive : (64, 256)
Linear : (64, 128)
BatchNorm1d : (64, 128)
Tanh : (64, 128)
Linear : (64, 27)


In [868]:
# put layers into eval mode (for batchnorm specially)
for layer in model.layers:
    layer.training = False

In [869]:
# evaluate the loss
@torch.no_grad()  # disables gradient tracking
def split_loss(split):
    x, y = {
        'train': (Xtr, Ytr),
        'val': (Xdev, Ydev),
        'test': (Xte, Yte),
    }[split]
    logits = model(x)
    loss = F.cross_entropy(logits, y)
    print(split, loss.item())

split_loss('train')
split_loss('val')

train 3.7331621646881104
val 3.7372987270355225


In [870]:
# sample from the model

for _ in range(20):
    
    out = []
    context = [0] * block_size  # initialize with all ...
    while True:
        # forward pass 
        logits = model(torch.tensor([context]))
        probs = F.softmax(logits, dim=1)

        # sample from the distribution
        ix = torch.multinomial(probs, num_samples=1).item()
        # shift the context window and track the samples
        context = context[1:] + [ix]
        out.append(ix)
        # if we sample '.' token, break
        if ix == 0:
            break

    print(''.join(itos[i] for i in out))  # decode and print generated word

ezbtbewwsxsuufurjlpt.
mjurhpagpjgwsagawwppjdjgyrgeasywhmveulwyluwpgsoabuvuwajruoy.
pemmw.
rpgyraspmyuo.
wsgupalurpocjaekalnsdugppyagzuzuubiryahhmwsiuwjypw.
wujshxuguwijrpgbkygwh.
akpodsauwsazsylykyuusspp.
grgappuyuewrrphyrtdwrsuwjowwpagapggjupspuwwpgbwwjpyw.
wsdpiugohlaxzgpgoerqotpz.
yefpspgpvptywuukwpp.
wgsawrsplryugwjtaalsgjblgpgjtsmpupgmpueaeogeozyodaoubpwbaavyisudyzuiuppywdkywmopipegmuguyidzgjrjpaaamwsedwbbzecmebubupfu.
wgmsjmgu.
ugasgsaprpaesyyyamhselposhcjwcuyeswjusmlaatwugjtwjubxjwiwacaxshspygteayespuyybyghuybhupkbxajgrrcu.
euuwjpgramraipyyuhwujdg.
yyaaeqppammuaggygppoyuyhuyfrpsuwposjppjkpwrppymhpgggeuavhwjwpgyglvpgjgwwfekeupuwuspzyzwhzfypsmj.
tfgypzwww.
jbawjauzhjkbiouwumujwccsewguj.
dpsulpghwzu.
eywyilcpbpkpgeybywuyupgwujgcugefzup.
gaupsuphvahuoorlyfgyrghmabunojdmpeajbbjgputgjwyywius.
